In [2]:
import pandas as pd
import numpy as np
import os
from ipywidgets import IntProgress
from IPython.display import display

In [32]:
dirList = os.listdir("data/Chunks/")
fList = []
for filename in dirList: #limit to .csv
    if(filename[-4:] == '.csv'):
        fList.append(filename[:-4]) 

In [35]:
for fName in fList[:]:
    print(fName)
    sDF = pd.read_csv('data/Chunks/'+fName+'.csv').drop(columns={'Unnamed: 0'})
    data = []
    for c in sDF.ContestId.unique()[:]:
        contestDF = sDF[sDF['ContestId']==c]
        val = contestDF[contestDF['SecondsRemaining']>240]['Entries'].sum()
        thisD = {'ContestId':c, 'FourHoursOut':val}
        data.append(thisD)
    thisChunk = pd.DataFrame(data=data)
#     print("---Saving"+'data/Chunk_Change/'+fName+'.csv')
    thisChunk.to_csv('data/Chunk_Change/'+fName+'.csv')

In [36]:
dirList = os.listdir("data/Chunks/")
fList = []
for filename in dirList: #limit to .csv
    if(filename[-4:] == '.csv'):
        fList.append(filename[:-4]) 

In [71]:
contests = pd.read_csv('data/WorkingData.csv').set_index('ContestId')[['Entries', 'MaxNumberPlayers']]

In [72]:
contests.head(1)

,Entries,MaxNumberPlayers
ContestId,,
7963004,117500.0,117500


In [77]:
full="None"
for fName in fList[:]:
    fContent = pd.read_csv('data/Chunk_Change/'+fName+'.csv').set_index('ContestId').drop(columns=['Unnamed: 0'])
    fContent.head(1)
    nextC = pd.merge(contests, fContent, on='ContestId', how='right')
    if(full is "None"):
        full = nextC
    else:
        full = pd.concat([full, nextC])


In [90]:
newC = full.dropna()
newC = newC.assign(ZeroHoursOut_Scaled=newC['Entries']/newC['MaxNumberPlayers'])
newC = newC.assign(FourHoursOut_Scaled=newC['FourHoursOut']/newC['MaxNumberPlayers'])
newC = newC.drop(columns=['Entries', 'FourHoursOut'])

In [94]:
noIndex = newC.reset_index().drop(columns=['ContestId'])

In [98]:
noIndex = noIndex[noIndex['FourHoursOut_Scaled']>0]

In [99]:
noIndex.to_csv('data/BrownData.csv')